In [2]:
base_path = 'source'
filename = 'country_list.csv'
storage_account_name = 'synapsesharingstorage'
container_name = 'root'
database = 'default'
target_table = '{"name":"country_list","path":"/data"}'
module_name = 'md5'
module_config = '{}'
pipeline_run_id = '123'

StatementMeta(SparkPool1, 13, 2, Finished, Available)

In [3]:
import importlib
import json
import logging 

logger = sc._jvm.org.apache.log4j.LogManager.getLogger("edm-log4j-logger") 

tranformer_module = importlib.import_module(module_name)

# log input parameters
logger.info(f"Pipeline Run ID : {pipeline_run_id}")
logger.debug(f"Base path parameter value {base_path}")
logger.debug(f"File name parameter value {filename}")
logger.debug(f"Storage account name parameter value {storage_account_name}")
logger.debug(f"Container Name parameter value {container_name}")
logger.debug(f"Database name parameter value {database}")
logger.debug(f"Target table parameter value {target_table}")
logger.debug(f"Module name parameter value {module_name}")
logger.debug(f"Target table parameter value {module_config}")

target_table_config = json.loads(target_table)
tranformation_config = json.loads(module_config)
# Parse
destination_table = target_table_config["name"]
external_table_path = target_table_config["path"]


# Read data from source, filename empty means data will be read by transformers internally directly from tables.  # noqa: E501
if (filename):
    adlsUri = f"abfss://{container_name}@{storage_account_name}.dfs.core.windows.net/"  # noqa: E501
    full_path = adlsUri + base_path
    # MVO-0 Works with Parquet files ONLY
    sourcedf = spark.read.option("header","true").csv(full_path + "/" + filename)
else:
    sourcedf = []

# Tranform the data
finaldf = tranformer_module.transform(spark, sourcedf, tranformation_config)
# Check for table existence
spark_tables = spark.catalog.listTables()
is_table_exists = any(x.name == destination_table for x in spark_tables)
if is_table_exists:
    finaldf.write.mode("append").insertInto(database + '.' + destination_table)
    logger.info(f"Data appended to table: {destination_table} after {module_name} transformation")
else:
    finaldf.write.mode("overwrite").option("path",external_table_path).saveAsTable(database + '.' + destination_table);
    logger.info(f"New table {destination_table} created after {module_name} transformation!")

mssparkutils.notebook.exit(filename)

StatementMeta(SparkPool1, 13, 3, Finished, Available)

ExitValue: country_list.csv